## Primeiro começamos por importar as bibliotecas necessárias para gerar os gráficos.

In [ ]:
import matplotlib.pyplot as plt
import random
import json
import math
from PIL import Image, ImageDraw, ImageFont

## Caso sua isntalação do Python não consiga importar as bibliotécas, use o PIP para fazê-lo. A bibliotéca PIL é deprecada, utilize a Pillow na instalação.

## Definindo o teclado

Comece criando um dict para guardar as coordenadas de cada tecla. A órdem que definimos para nosso projeto foi de cima para baixo, esquerda para direita. Isso quer dizer que num teclado de layout QWERTY, a letra "Q" corresponde ao índice 0, "A" é 1, "Z" é 2, "W" é 3, e assim sucessivamente. A linha central e inferior do teclado, serão descoladas em relação a linha superior, caso queiramos um teclado ortolinear, devemos inicar o Offset em 0.

In [ ]:
KEY_WIDTH = 94
MIDDLE_OFFSET = 24
BOTTOM_OFFSET = 71
offsets = [0, MIDDLE_OFFSET, BOTTOM_OFFSET]

coords = {}
for i in range(30):
    row = i%3
    column = math.floor(i/3)
    x = column*KEY_WIDTH + offsets[row]
    y = row*KEY_WIDTH
    coords[i] = (x, y)
coords

## Teclas por dedo.

Defina o dedo responsável por cada tecla. Isso pode ser alterarado dependendo de como quisermos simular a digitação. Por exemplo assumindo que devemos digitar com os 10 dedos, o dedo médio, é responsável pelas teclas "W", "S", "X". Portanto os índices 0, 1 e 2 serão agrupados num array bidimensional. Nós definimos as teclas de cada dedo como elementos no array bidimensional.

In [ ]:
keys_per_finger = [[0,1,2], [3,4,5], [6,7,8], [9,10,11,12,13,14], [15,16,17,18,19,20], [21,22,23], [24,25,26], [27,28,29]]
# As teclas definidas para digitação com dois dedos seria a seguinte:
# [[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14], [15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]]
# As teclas definidas para digitação com um dedo seria a seguinte:
# [[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]]

## Teclas centrais

Ao digitar, cada dedo que é utilizado deve ter uma tecla central que é sua posição 0 a qual eles começam e devem retornar quando não estão digitando. No teclado QWERTY por exemplo as teclas centrais de início, são: "A"S"D"F"J"K"L"Ç". No teclado americano que é que nós utilizamos nesse trabalho por ser mais comun, tem no lugar do "Ç" o ";". Isso corresponde aos índices 1, 4, 7, 10, 19, 22, 25, 28. Use um dict para associar cada tecla a seu índice de posição central.

In [ ]:
home_key_pos = [1, 4, 7, 10, 19, 22, 25, 28]
# A posição central para digitação de dois dedos é a seguinte:
# home_key_pos = [7, 22]
# A posição central para digital com apenas um dedo é a seguinte:
# home_key_pos = [16]

home_keys = {}
for i, keys in enumerate(keys_per_finger):
    for key in keys:
        home_keys[key] = home_key_pos[i]
home_keys

## Objeto teclado

Cada teclado será definido por um dict de quais caracteres cada tecla representa. Essa função converterá um genoma (Uma string que representa um teclado) em um dict. O genoma deve estar em ordem, Então para o QWERTY, o genoma será "qazwsxedcrfvtgbyhnujmik,ol.p;/". Também levará em consideração as teclas de câmbio como "<", ">", ":", and "?", adicionando-as ao dict com a tecla correspondente.

In [ ]:
def genome_to_keyboard(genome):
    keyboard = {}
    for i, char in enumerate(genome):
        keyboard[char] = i
        if char == ',':
            keyboard['<'] = i
        elif char == '.':
            keyboard['>'] = i
        elif char == ';':
            keyboard[':'] = i
        elif char == '/':
            keyboard['?'] = i
    return keyboard

## Dados    

### Dataser arXiv.org

O conjunto de dados de metadados arXiv.org pode ser baixado de [Kaggle](https://www.kaggle.com/datasets/Cornell-University/arxiv). Baixe o arquivo json e salve-o no mesmo diretório deste notebook. O código a seguir criará uma coleção de resumos. Não incluirá resumos que contenham caracteres ilegais. Isso reduzirá quaisquer dados indesejados, como notação científica e equações matemáticas. No vídeo original, apenas um pequeno subconjunto do conjunto de dados de 587 resumos. O conjunto de dados é muito grande e calcular a distância em todo o conjunto de dados levará muito tempo.


In [ ]:
ARXIV_JSON = '../arxiv-metadata-oai-snapshot.json'
DATA_LIMIT = 1000

full_text = ''
legal_chars = 'qazwsxedcrfvtgbyhnujmik,ol.p;:? '

count = 0
with open(ARXIV_JSON) as file:
    for line in file:
        if count > DATA_LIMIT:
            break
        abstract = json.loads(line)['abstract'].replace('\n', ' ').strip().lower()
        if any(char not in legal_chars for char in abstract):
            continue
        full_text += ' ' + abstract
        count += 1

## Calculando a distância

### Calculando a distância entre duas teclas

Dada duas teclas, a função abaixo irá calcular a ditância relativa entre elas.

In [ ]:
def distance(first, second):
    return math.hypot(second[0] - first[0], second[1] - first[1])

### Calculando a distância de todos os pares de letras

Calcule a distância para todos os pares de letras válidos e salve-a como um dicionário de dicionários. A chave no dicionário corresponde à chave inicial. A chave do dicionário interno é a chave final e o valor do dicionário interno é a distância desse par de chaves. Por exemplo, as distâncias da chave 0 às chaves 0, 1 e 2 serão representadas como:
```
{
  0: {
    0: 0.0, 
    1: 1.0320793902668004, 
    2: 2.1378744155702085
  }
}
```

In [ ]:
distances = {i: {} for i in range(30)}
for keys in keys_per_finger:
    for i in keys:
        for j in keys:
            distances[i][j] = distance(coords[i], coords[j]) / KEY_WIDTH
distances

## Calculando a distância total de uma determinada string

Essa função irá calcular a distância total de qualquer texto dado qualquer teclado.

In [ ]:
def total_distance(input_string, keyboard):
    input_string = input_string.lower()
    input_string = input_string.replace(' ', '')
    first_char = input_string[0]
    first_pos = keyboard[first_char]
    first_home_key = home_keys[first_pos]
    total_dist = distances[first_home_key][first_pos]
    for i in range(0, len(input_string)-1):
        cur_char = input_string[i]
        next_char = input_string[i+1]
        cur_pos = keyboard[cur_char]
        next_pos = keyboard[next_char]
        if cur_pos in distances and next_pos in distances[cur_pos]:
            total_dist += distances[cur_pos][next_pos]
        else:
            home_key = home_keys[next_pos]
            total_dist += distances[home_key][next_pos]
    return total_dist

## Testando a distância.

Calculando a distância em um frase de teste.

In [ ]:
test_string = 'O menino chutou a bola na janlea e quebrou o vidro.'
qwerty = genome_to_keyboard(list('qazwsxedcrfvtgbyhnujmik,ol.p;/'))
total_distance(test_string, qwerty)

### Calculando a distância total do dataset.

In [ ]:
total_distance(full_text, qwerty)

## Altoritmo genético.

### Inicialize a população.

A função abaixo irá inicializar a população para a primeira geração com teclados randomicos dado um tamanho de população.

In [ ]:
def init_population(pop_size):
    keyboard_chars = list('qazwsxedcrfvtgbyhnujmik,ol.p;/')
    population = []
    for i in range(pop_size):
        rand_gnome = keyboard_chars[:]
        random.shuffle(rand_gnome)
        population.append(rand_gnome)
    return population

## Combiando dois teclados.

Esta função define a lógica de "cruzamento" de dois teclados para criar um novo teclado. A função selecionará um ponto aleatório para dividir os teclados. Ele começará a preencher o teclado filho à direita do ponto de divisão com um número aleatório de teclas do primeiro teclado. Em seguida, ele preencherá as teclas restantes com as teclas do teclado 2. Há também uma chance aleatória de mutação em que duas teclas do teclado filho trocarão de lugar.

In [ ]:
def mate(board1, board2, mutation_rate):
    keyboard_size = len(board1)
    idx = random.randint(0, keyboard_size-1)
    length = random.randint(0,keyboard_size-1)
    child = ['_' for i in range(keyboard_size)]
    for i in range(length):
        if idx > keyboard_size-1:
            idx = 0
        child[idx] = board1[idx]
        idx += 1
    child_idx = idx
    while '_' in child:
        if idx > keyboard_size-1:
            idx = 0
        if child_idx > keyboard_size-1:
            child_idx = 0
        char = board2[idx]
        if char in child:
            idx += 1
            continue
        child[child_idx] = board2[idx]
        child_idx += 1
        idx += 1
        
    prob = random.random()
    if prob < mutation_rate:
        point1 = random.randint(0, 29)
        point2 = random.randint(0, 29)
        allele1 = child[point1]
        allele2 = child[point2]
        child[point1] = allele2
        child[point2] = allele1
        
    return child

## Avalianado a população.

Essa função avaliará uma determinada população calculando a distância total para cada teclado na população. Ele retorna as avaliações como um dicionário. Ele também retorna os índices classificados na ordem da distância total do teclado naquele índice.

In [ ]:
def get_evals(population):
    evals = {}
    for i, genome in enumerate(population):
        keyboard = genome_to_keyboard(genome)
        dist = total_distance(full_text, keyboard)
        evals[i] = dist
    sorted_evals = [k for k, v in sorted(evals.items(), key=lambda item: item[1])]
    return evals, sorted_evals

## Criando a próxima geração.

Essa função criará a próxima geração utilizando a população atual. Ela copiará diretamente os top 10% teclados da população anterior para a próxima e utilizará os top 50% para criar o resto da próxima geração.

In [ ]:
def new_generation(population, sorted_evals, p_size, mutation_rate):
    new_gen = []
    
    sorted_population = []
    for i in sorted_evals:
        sorted_population.append(population[i])
        
    for i in range(int(p_size*0.1)):
        new_gen.append(sorted_population[i])

    for _ in range(int(p_size*0.9)):
        p1 = random.choice(sorted_population[:int(p_size*0.5)])
        p2 = random.choice(sorted_population[:int(p_size*0.5)])
        child = mate(p1, p2, mutation_rate)
        new_gen.append(child)
    
    return new_gen

## Executando o algoritmo.

O código a seguir executará o algoritmo genético. Ajuste as constantes `P_SIZE` para alterar o tamanho da população, `GENERATIONS` para alterar o número total de gerações que o algoritmo executará e `MUTATION_RATE` para alterar a frequência com que as mutações ocorrem durante o acasalamento. Os dados de treinamento serão armazenados no dicionário `learning` e salvos em um arquivo json assim que o algoritmo for concluído. Isso conterá informações para cada geração. Ele conterá o total de todos os teclados da população, o melhor teclado da população, a menor distância do melhor teclado e a distância média de todos os teclados.

In [ ]:
P_SIZE = 100
GENERATIONS = 300
MUTATION_RATE = .1

learning = {
    'generations': {}
}

population = init_population(P_SIZE)

for i in range(GENERATIONS):    
    evals, sorted_evals = get_evals(population)
    sum_evals = 0
    for key in evals:
        sum_evals += evals[key]
    avg_evals = sum_evals/P_SIZE
    learning['generations'][i] = {
        'population': population,
        'best': population[sorted_evals[0]],
        'min': evals[sorted_evals[0]],
        'avg': avg_evals
    }
    print('GEN: {}, AVG: {}, MIN: {}, BEST: {}'.format(i+1, avg_evals, evals[sorted_evals[0]], population[sorted_evals[0]]))
    
    population = new_generation(population, sorted_evals, P_SIZE, MUTATION_RATE)

LEARNING_JSON = 'learning.json'
with open(LEARNING_JSON, 'w') as fp:
    json.dump(learning, fp)

## Visualizando o trainamento.

O código a baixo utilizará o arquivo json com o treinamento e fará a plotagem dos dados.
Um gráfico para a média de distância das gerações e outro para a menor distância por geração.

In [ ]:
## A linha abaixo serve para visualizar o gráfico das distâncias sem precisar rodar o notebook por completo.
## LEARNING_JSON = 'treino1.json'

with open(LEARNING_JSON) as fp:
    learning = json.load(fp)
    
last_dist = 1000000000
min_dists = []
avg_dists = []
generations = len(learning['generations'])

for i in range(0, generations):
    min_dist = learning['generations'][str(i)]['min']
    avg_dist = learning['generations'][str(i)]['avg']
    min_dists.append(min_dist)
    avg_dists.append(avg_dist)

plt.plot(min_dists, label='Lowest Distance')
plt.xlabel('Generations')
plt.ylabel('Distance')
plt.legend()
plt.show()

plt.plot(avg_dists, label='Average Distance', color='orange')
plt.xlabel('Generations')
plt.ylabel('Distance')
plt.legend()
plt.show()

## Visualizando os layouts

O código abaixo cria uma imagem do melhor teclado ao final do treinamento.

In [ ]:
# O código comentado abaixo serve para visualizar o melhor teclado sem precisar rodar o notebook por completo
# LEARNING_JSON = 'treino1.json'
# GENERATIONS = 300

# with open(LEARNING_JSON) as fp:
#     learning = json.load(fp)

kb = learning['generations'][str(GENERATIONS-1)]['best'] # best keyboard found

with Image.open("template.jpg").convert("RGBA") as base:

    # make a blank image for the text, initialized to transparent text color
    txt = Image.new("RGBA", base.size, (255, 255, 255, 0))

    # get a font
    fnt = ImageFont.truetype("SFNSMono.ttf", 40)
    # get a drawing context
    d = ImageDraw.Draw(txt)
    
    x_offsets = [110, 135, 175]
    for i in range(30):
        row = i%3
        column = math.floor(i/3)
        x = column*60 + x_offsets[row]
        y = row*65 + 85
        char_coords = (x, y)
        d.text(char_coords, kb[i], font=fnt, fill=(0, 0, 0, 255))

    out = Image.alpha_composite(base, txt)

    display(out)